In [10]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from web3 import Web3
import json
import time

# Load environment variables
load_dotenv()

# Connect to the Base network
#w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))
w3 = Web3(Web3.HTTPProvider('https://base-mainnet.infura.io/v3/26DtrLheKXrxF7h7gWZ2N2fYUVw'))
# Load contract ABIs
with open('AuctionHouse_ABI.json', 'r') as abi_file:
    auction_house_abi = json.load(abi_file)
    
with open('AuctionHouseFactory_ABI.json', 'r') as abi_file:
    auction_house_factory_abi = json.load(abi_file)
    
with open('AffiliateEscrow_ABI.json', 'r') as abi_file:
    affiliate_escrow_abi = json.load(abi_file)
    
    
with open('AuctionItemERC721_ABI.json', 'r') as abi_file:
    auction_item_erc721_abi = json.load(abi_file)
    
with open('AuctionItemERC721Factory_ABI.json', 'r') as abi_file:
    auction_item_erc721_factory_abi = json.load(abi_file)
    
with open('ERC20_ABI.json', 'r') as abi_file:
    erc20_abi = json.load(abi_file)

# Set up contract addresses
AUCTION_HOUSE_FACTORY_ADDRESS = os.getenv('AUCTION_HOUSE_FACTORY_ADDRESS')
AUCTION_ITEM_ERC721_FACTORY_ADDRESS = os.getenv('AUCTION_ITEM_ERC721_FACTORY_ADDRESS')

# Set up contract instances
auction_house_factory = w3.eth.contract(
    address=AUCTION_HOUSE_FACTORY_ADDRESS, 
    abi=auction_house_factory_abi
)


auction_item_erc721_factory = w3.eth.contract(
    address=AUCTION_ITEM_ERC721_FACTORY_ADDRESS, 
    abi=auction_item_erc721_factory_abi
)

# Set up accounts
owner_account = w3.eth.account.from_key(os.getenv('OWNER_PRIVATE_KEY'))
seller_account = w3.eth.account.from_key(os.getenv('SELLER_PRIVATE_KEY'))
bidder1_account = w3.eth.account.from_key(os.getenv('BIDDER1_PRIVATE_KEY'))
bidder2_account = w3.eth.account.from_key(os.getenv('BIDDER2_PRIVATE_KEY'))
affiliate_account = w3.eth.account.from_key(os.getenv('AFFILIATE_PRIVATE_KEY'))
arbiter_account = w3.eth.account.from_key(os.getenv('ARBITER_PRIVATE_KEY'))

# Example encrypted message structure for bids
EXAMPLE_MESSAGE = {
    "encryptedData": "0x",
    "ephemeralPublicKey": "0x",
    "iv": "0x",
    "verificationHash": "0x"
}

# Constants for the auction system
DEFAULT_SETTLEMENT_DEADLINE = 21 * 24 * 60 * 60  # 21 days in seconds

In [11]:
owner_account.address

'0x9f4640d04371ff6b7886ade5323746388107723a'

In [39]:
def create_auction_house(name, image, description, sender_account, custom_deadline=0):
    """Create a new auction house using the factory"""
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    # Get the addresses for required factories
    auction_item_factory_address = AUCTION_ITEM_ERC721_FACTORY_ADDRESS
    escrow_factory_address = os.getenv('AFFILIATE_ESCROW_FACTORY_ADDRESS')
    
    # Updated parameters to match the new contract interface
    txn = auction_house_factory.functions.createAuctionHouse(
        name,
        image,
        description,
        "https://example.com/contract-metadata",  # Contract URI
        "AUCTN",  # Symbol
        custom_deadline,
        auction_item_factory_address,
        escrow_factory_address
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 69420000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process logs to get the new auction house address
    for log in tx_receipt['logs']:
        try:
            event = auction_house_factory.events.AuctionHouseCreated().process_log(log)
            if event:
                return event['args']['auctionHouse']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionHouseCreated event found in the transaction logs")
    return None

def create_auction_with_new_nft(auction_house_contract, metadata, start_time, reserve_price, 
                               duration, affiliate_fee, arbiter_address,
                               escrow_factory_address=None, payment_token=None, 
                               premium=False, premium_rate=0, min_bid_increment=500, 
                               time_extension=900):
    """Create a new auction with a new NFT in one transaction"""
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
    
    # If escrow factory not provided, get it from the auction house
    if escrow_factory_address is None:
        escrow_factory_address = auction_house_contract.functions.escrowFactory().call()
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    print("\n=== Creating Auction with New NFT ===")
    print(f"Start Time: {start_time} ({time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))})")
    print(f"Reserve Price: {w3.from_wei(reserve_price, 'ether')} ETH")
    print(f"Duration: {duration} seconds ({duration/86400:.2f} days)")
    print(f"Affiliate Fee: {affiliate_fee} basis points ({affiliate_fee/100:.2f}%)")
    
    # Build the transaction
    txn = auction_house_contract.functions.createAuctionWithNewNFT(
        metadata,
        start_time,
        reserve_price,
        duration,
        affiliate_fee,
        arbiter_address,
        escrow_factory_address,
        payment_token,
        premium,
        premium_rate,
        min_bid_increment,
        time_extension
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 6000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the auction ID
    auction_id = None
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.AuctionCreated().process_log(log)
            if event:
                auction_id = event['args']['auctionId']
                break
        except Exception as e:
            continue
    
    if auction_id is None:
        print("No AuctionCreated event found in the transaction logs")
        return None
    
    # Get the auction data to find the token ID
    auction_data = auction_house_contract.functions.auctions(auction_id).call()
    token_id = auction_data[0]  # Index of tokenId in the Auction struct
    
    return {
        "auction_id": auction_id,
        "token_id": token_id,
        "tx_receipt": tx_receipt
    }


def create_nft_contract_via_auction_house(auction_house_contract, name, symbol, contract_uri):
    """Create a new NFT contract through the auction house"""
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = auction_house_contract.functions.createNFTContract(
        name,
        symbol,
        contract_uri
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the NFT contract address from the mapping
    nft_contract_address = auction_house_contract.functions.nftContracts(symbol).call()
    print(f"Created new NFT contract via auction house at: {nft_contract_address}")
    return nft_contract_address

# Update the mint_auction_item function to work with the auction house
def mint_auction_item(token_address, metadata, to_address=None):
    """Mint a new token with metadata and return its ID"""
    # Create contract instance
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    # Use provided address or default to owner account
    mint_to_address = to_address if to_address else owner_account.address
    
    # Mint token to the specified address
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = token_contract.functions.mintWithMetadata(
        mint_to_address,
        metadata["name"],
        metadata["description"],
        metadata["image"],
        metadata["termsOfService"],
        metadata.get("supplementalImages", [])
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the token ID from the event
    token_id = None
    for log in tx_receipt['logs']:
        try:
            event = token_contract.events.Transfer().process_log(log)
            if event and event['args']['from'] == '0x0000000000000000000000000000000000000000':
                token_id = event['args']['tokenId']
                print(f"Minted token ID: {token_id} to address: {mint_to_address}")
                break
        except Exception as e:
            continue
    
    if token_id is None:
        print("Failed to get token ID from mint transaction")
        return None
    
    return token_id

# Add a function to approve the auction house to transfer the NFT
def approve_nft_for_auction(token_address, token_id, auction_house_address):
    """Approve the auction house to transfer the NFT"""
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    txn = token_contract.functions.approve(
        auction_house_address,
        token_id
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 200000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Approved auction house {auction_house_address} to transfer NFT {token_id}")
    return tx_receipt

# Update the create_auction function to work with the new parameters
def create_auction(auction_house_contract, token_address, token_id, start_time, reserve_price, 
                  duration, affiliate_fee, arbiter_address, 
                  payment_token=None, premium=False, premium_rate=0, 
                  min_bid_increment=500, time_extension=900):
    """Create a new auction"""
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
    
    # Get the escrow factory address from the auction house
    escrow_factory_address = auction_house_contract.functions.escrowFactory().call()
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    txn = auction_house_contract.functions.createAuction(
        token_address,
        token_id,
        start_time,
        reserve_price,
        duration,
        affiliate_fee,
        arbiter_address,
        escrow_factory_address,
        payment_token,
        premium,
        premium_rate,
        min_bid_increment,
        time_extension
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the auction ID
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.AuctionCreated().process_log(log)
            if event:
                return event['args']['auctionId']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionCreated event found in the transaction logs")
    return None
def get_auction_data(auction_house_contract, auction_id):
    """
    Get detailed data for a specific auction
    
    Args:
        auction_house_contract: Web3 contract instance of the auction house
        auction_id: ID of the auction to query
        
    Returns:
        Dictionary with formatted auction data
    """
    try:
        # Call the getAuctionData function
        auction_data = auction_house_contract.functions.getAuctionData(auction_id).call()
        
        # Return the data with correct field names based on the contract struct
        return {
            "tokenId": auction_data[0],
            "tokenContract": auction_data[1],
            "highestBid": auction_data[2],
            "endTime": auction_data[3],
            "startTime": auction_data[4],
            "reservePrice": auction_data[5],
            "affiliateFee": auction_data[6],
            "auctionOwner": auction_data[7],
            "bidder": auction_data[8],
            "affiliate": auction_data[9],
            "arbiter": auction_data[10],
            "escrowAddress": auction_data[11],
            "auctionCurrency": auction_data[12],
            "minBidIncrementBps": auction_data[13],
            "isPremiumAuction": auction_data[14],
            "premiumBps": auction_data[15],
            "timeExtension": auction_data[16],
            "paymentAmount": auction_data[17]
        }
    except Exception as e:
        print(f"Error getting auction data: {str(e)}")
        return None
    
def end_auction(auction_house_contract, auction_id, sender_account):
    """
    End an auction
    
    Args:
        auction_house_contract: Web3 contract instance of the auction house
        auction_id: ID of the auction to end
        sender_account: Account to send the transaction from
        
    Returns:
        Transaction receipt
    """
    try:
        nonce = w3.eth.get_transaction_count(sender_account.address)
        
        print(f"Ending Auction {auction_id}...")
        
        # Build the transaction
        txn = auction_house_contract.functions.endAuction(
            auction_id
        ).build_transaction({
            'from': sender_account.address,
            'nonce': nonce,
            'gas': 500000,
            'gasPrice': w3.eth.gas_price
        })
        
        signed_txn = sender_account.sign_transaction(txn)
        tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        print(f"Transaction sent: {tx_hash.hex()}")
        
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Transaction confirmed in block {tx_receipt['blockNumber']}")
        
        return tx_receipt
        
    except Exception as e:
        print(f"Error ending auction: {str(e)}")
        return None
# Add a function to settle the escrow after auction ends
def settle_escrow(escrow_address, token_address=None):
    """Settle the escrow after auction ends"""
    # Create escrow contract instance
    escrow_contract = w3.eth.contract(
        address=escrow_address,
        abi=affiliate_escrow_abi
    )
    
    # Get the payer (winning bidder)
    payer = escrow_contract.functions.payer().call()
    
    # Create a transaction to settle the escrow
    # Use the bidder account that matches the payer
    if payer == bidder1_account.address:
        bidder_account = bidder1_account
    elif payer == bidder2_account.address:
        bidder_account = bidder2_account
    else:
        print(f"Unknown payer address: {payer}")
        return None
    
    nonce = w3.eth.get_transaction_count(bidder_account.address)
    
    # Get the escrow balance
    if token_address is None or token_address == "0x0000000000000000000000000000000000000000":
        token_address = "0x0000000000000000000000000000000000000000"  # ETH
        escrow_balance = w3.eth.get_balance(escrow_address)
    else:
        token_contract = w3.eth.contract(address=token_address, abi=erc20_abi)
        escrow_balance = token_contract.functions.balanceOf(escrow_address).call()
    
    txn = escrow_contract.functions.settle(
        token_address,
        escrow_balance
    ).build_transaction({
        'from': bidder_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = bidder_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Settled escrow at {escrow_address}")
    return tx_receipt

In [13]:
AUCTION_HOUSE_FACTORY_ADDRESS

'0x8279B3D660453cbbf46B232b439A8Ab1cA647fa8'

In [14]:
# Step 1: Create a new auction house
auction_house_address = create_auction_house(
    "Test Auction House",
    "https://example.com/image.png",
    "A test auction house for NFTs",
    owner_account,  # Added the sender account parameter
    30 * 24 * 60 * 60  # 30 days settlement deadline
)

print(f"New auction house created at: {auction_house_address}")

# Create auction house contract instance
auction_house = w3.eth.contract(
    address=auction_house_address,
    abi=auction_house_abi
)

Transaction receipt: AttributeDict({'blockHash': HexBytes('0xc9a7bd789f5c6bca0d9667c54486e12820f1bc1eed5ef99d667958f2de2994cd'), 'blockNumber': 29880001, 'contractAddress': None, 'cumulativeGasUsed': 58307310, 'effectiveGasPrice': 1142878, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 5871497, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x1f3e2b33d', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x20b017d4285', 'l1GasPrice': '0x393c4031', 'l1GasUsed': '0xfcb', 'logs': [AttributeDict({'address': '0xfda91e4F38dec9D5B45e4F013B1e7C85a57020A3', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x0000000000000000000000008279b3d660453cbbf46b232b439a8ab1ca647fa8')], 'data': HexBytes('0x'), 'blockNumber': 29880001, 'transactionHash': HexBytes('0x8bfc23c542486be9762b93ff6cbe2d31e0fc1a3d369de42677f7437802a825f7'), 'transactionIndex': 113, 'blockHas

In [21]:
auction_house.address

'0xfda91e4F38dec9D5B45e4F013B1e7C85a57020A3'

In [36]:
# Get auction data
auction_info = get_auction_data(auction_house, 0)
print(f"Auction info: {auction_info}")

Auction info: {'tokenId': 1, 'tokenContract': '0xb9DF696e94521919B6Dd2bb820B5fD6fe7B24533', 'highestBid': 602002501010, 'endTime': 1746550389, 'startTime': 1746549431, 'reservePrice': 1000, 'affiliateFee': 500, 'auctionOwner': '0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B', 'bidder': '0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA', 'affiliate': '0x31D96034FC743CEA862e9C2D0A85747dE355e0f7', 'arbiter': '0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B', 'escrowAddress': '0x2c394AE8A0D7Ca315830452fb5E550929DC1eF84', 'auctionCurrency': '0x0000000000000000000000000000000000000000', 'minBidIncrementBps': 50, 'isPremiumAuction': True, 'premiumBps': 10000, 'timeExtension': 900, 'paymentAmount': 600500001005}


In [15]:
# Complete test flow for creating an auction with a new NFT
# Step 1: Set up parameters
current_time = int(time.time())
start_time = current_time + 60  # Start in 1 minute
reserve_price = w3.to_wei(0.000000000000001, 'ether')  # 0.01 ETH
duration = 5 * 60  # 5 minutes for testing
affiliate_fee = 500  # 5%

nft_metadata = {
    "name": "Auction Item #1",
    "description": "This is a unique digital item being auctioned",
    "image": "https://assets.surlatable.com/m/15a89c2d9c6c1345/72_dpi_webp-REC-283110_Pizza.jpg",
    "termsOfService": "By bidding on this item, you agree to the terms of service.",
    "supplementalImages": ["https://assets.surlatable.com/m/15a89c2d9c6c1345/72_dpi_webp-REC-283110_Pizza.jpg", "https://assets.surlatable.com/m/15a89c2d9c6c1345/72_dpi_webp-REC-283110_Pizza.jpg"]
}

# Get the escrow factory address
escrow_factory_address = os.getenv('AFFILIATE_ESCROW_FACTORY_ADDRESS')

# Step 2: Create the auction with a new NFT
print("Creating auction with new NFT...")
result = create_auction_with_new_nft(
    auction_house,
    nft_metadata,
    start_time,
    reserve_price,
    duration,
    affiliate_fee,
    arbiter_account.address,
    escrow_factory_address,
    None,  # payment_token (default to ETH)
    True,  # premium
    10000,      # premium_rate
    50,    # min_bid_increment (5%)
    900     # time_extension (15 minutes)
)

if result:
    auction_id = result["auction_id"]
    token_id = result["token_id"]
    print(f"Created auction with ID {auction_id} for NFT with token ID {token_id}")
else:
    print("Failed to create auction")

Creating auction with new NFT...

=== Creating Auction with New NFT ===
Start Time: 1746549431 (2025-05-06 16:37:11)
Reserve Price: 1E-15 ETH
Duration: 300 seconds (0.00 days)
Affiliate Fee: 500 basis points (5.00%)
Created auction with ID 0 for NFT with token ID 1


Utility functions for placing bids

In [16]:
def place_bid(auction_house_contract, auction_id, bidder_account, affiliate_address, bid_amount):
    """Place a bid on an auction"""
    # Convert affiliate_address to checksum
    affiliate_address = w3.to_checksum_address(affiliate_address)
    
    # Create empty encrypted message
    empty_message = {
        "encryptedData": "0x",
        "ephemeralPublicKey": "0x",
        "iv": "0x",
        "verificationHash": "0x"
    }
    
    nonce = w3.eth.get_transaction_count(bidder_account.address)
    
    print(f"\n=== Placing Bid ===")
    print(f"Auction ID: {auction_id}")
    print(f"Bidder: {bidder_account.address}")
    print(f"Bid Amount: {w3.from_wei(bid_amount, 'ether')} ETH")
    print(f"Affiliate: {affiliate_address}")
    
    txn = auction_house_contract.functions.createBid(
        auction_id,
        affiliate_address,
        empty_message,
        bid_amount
    ).build_transaction({
        'from': bidder_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
        'value': bid_amount  # For ETH bids
    })
    
    signed_txn = bidder_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Check if the bid was successful by looking for the BidCreated event
    bid_successful = False
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.BidCreated().process_log(log)
            if event and event['args']['auctionId'] == auction_id:
                bid_successful = True
                print(f"✅ Bid placed successfully!")
                break
        except Exception as e:
            continue
    
    if not bid_successful:
        print(f"❌ Bid placement may have failed. Check transaction: {tx_hash.hex()}")
    
    return tx_receipt

def get_minimum_bid(auction_house_contract, auction_id):
    """Get the minimum bid required for an auction"""
    try:
        min_bid = auction_house_contract.functions.getMinimumBid(auction_id).call()
        print(f"Minimum bid required: {w3.from_wei(min_bid, 'ether')} ETH")
        return min_bid
    except Exception as e:
        print(f"Error getting minimum bid: {str(e)}")
        # If we can't get the minimum bid, try to get the auction data directly
        try:
            auction = auction_house_contract.functions.auctions(auction_id).call()
            if auction[8] == '0x0000000000000000000000000000000000000000':  # No bidder yet
                return auction[2]  # Reserve price
            else:
                # Calculate based on highest bid and increment
                increment = (auction[2] * auction[13]) // 10000
                return auction[2] + increment
        except Exception as e2:
            print(f"Error getting auction data: {str(e2)}")
            return None

In [17]:
# Step 3: Wait for the auction to start
current_time = int(time.time())
wait_time = max(0, start_time - current_time) + 5  # Add 5 seconds buffer
print(f"Waiting for auction to start at {start_time} ({time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))})")
print(f"Current time: {current_time} ({time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time))})")
print(f"Waiting {wait_time} seconds...")
time.sleep(wait_time)

# Step 4: Bidder 1 places first bid
min_bid = get_minimum_bid(auction_house, auction_id)
if min_bid:
    bid1_amount = min_bid + w3.to_wei(0.0000001, 'ether')  # Add 0.001 ETH to minimum
    print(f"\nBidder 1 placing bid of {w3.from_wei(bid1_amount, 'ether')} ETH")
    place_bid(auction_house, auction_id, bidder1_account, affiliate_account.address, bid1_amount)
    
    # Step 5: Bidder 2 outbids Bidder 1
    time.sleep(10)  # Wait a bit between bids
    min_bid = get_minimum_bid(auction_house, auction_id)
    if min_bid:
        bid2_amount = min_bid + w3.to_wei(0.0000002, 'ether')  # Add 0.002 ETH to minimum
        print(f"\nBidder 2 placing bid of {w3.from_wei(bid2_amount, 'ether')} ETH")
        place_bid(auction_house, auction_id, bidder2_account, affiliate_account.address, bid2_amount)
        
        # Step 6: Bidder 1 outbids Bidder 2 (final bid)
        time.sleep(10)  # Wait a bit between bids
        min_bid = get_minimum_bid(auction_house, auction_id)
        if min_bid:
            bid3_amount = min_bid + w3.to_wei(0.0000003, 'ether')  # Add 0.003 ETH to minimum
            print(f"\nBidder 1 placing final bid of {w3.from_wei(bid3_amount, 'ether')} ETH")
            place_bid(auction_house, auction_id, bidder1_account, affiliate_account.address, bid3_amount)
else:
    print("Could not determine minimum bid amount")

Waiting for auction to start at 1746549431 (2025-05-06 16:37:11)
Current time: 1746549420 (2025-05-06 16:37:00)
Waiting 16 seconds...
Minimum bid required: 1E-15 ETH

Bidder 1 placing bid of 1.00000001E-7 ETH

=== Placing Bid ===
Auction ID: 0
Bidder: 0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA
Bid Amount: 1.00000001E-7 ETH
Affiliate: 0x31D96034FC743CEA862e9C2D0A85747dE355e0f7
✅ Bid placed successfully!
Minimum bid required: 1.00500001005E-7 ETH

Bidder 2 placing bid of 3.00500001005E-7 ETH

=== Placing Bid ===
Auction ID: 0
Bidder: 0x31D96034FC743CEA862e9C2D0A85747dE355e0f7
Bid Amount: 3.00500001005E-7 ETH
Affiliate: 0x31D96034FC743CEA862e9C2D0A85747dE355e0f7
✅ Bid placed successfully!
Minimum bid required: 3.0200250101E-7 ETH

Bidder 1 placing final bid of 6.0200250101E-7 ETH

=== Placing Bid ===
Auction ID: 0
Bidder: 0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA
Bid Amount: 6.0200250101E-7 ETH
Affiliate: 0x31D96034FC743CEA862e9C2D0A85747dE355e0f7
✅ Bid placed successfully!


In [40]:
#end the auction
end_auction(auction_house, auction_id, owner_account)


Ending Auction 0...
Transaction sent: 0x6480430a01b7101268af305b9c8c7a07d796c8960f844ea12694b722018a55b7
Transaction confirmed in block 29887333


AttributeDict({'blockHash': HexBytes('0x2ecab25e1e41d278c15433da531160c128df39eeff4ef25816ed73472b0a8fe4'),
 'blockNumber': 29887333,
 'contractAddress': None,
 'cumulativeGasUsed': 29335768,
 'effectiveGasPrice': 1161838,
 'from': '0x9f4640d04371ff6b7886ade5323746388107723a',
 'gasUsed': 183256,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x182f3',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x7b092079',
 'l1GasPrice': '0x21b7fb4f',
 'l1GasUsed': '0x640',
 'logs': [],
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [8]:
# Step 2: Create an NFT contract for multiple auctions
x = create_auction_item_erc721(
    "Test NFT",
    "TEST",
    "https://example.com/nft.json",
    owner_account  # Add the sender account parameter
)

print(f"New NFT contract created at: {x}")

Transaction receipt logs:
Log 0: AttributeDict({'address': '0xC9b577fE25018CBc12fcee1650c3551Aca21B2F9', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb')], 'data': HexBytes('0x'), 'blockNumber': 29534012, 'transactionHash': HexBytes('0x36c88ca95b94b08389e2add8039a7e8bc36773aeb7fed6917fa0c507c07c7c07'), 'transactionIndex': 190, 'blockHash': HexBytes('0x705bb0578314d7e7b2f59961660b0894ecfa6238a47116631a273e1222d20dd3'), 'logIndex': 508, 'removed': False})
Log 1: AttributeDict({'address': '0xC9b577fE25018CBc12fcee1650c3551Aca21B2F9', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb'), HexBytes('0x0000000000000000000000009f4640d04371ff6b7886ade5323746388107723a')], 'data': He

In [11]:
# Step 3: Mint the item to the auction house
token_id = mint_auction_item(
    x,  # NFT contract address
    {
        "name": "Test Item", 
        "description": "A test item", 
        "image": "https://example.com/nft.png", 
        "termsOfService": "https://example.com/terms.txt"
    },
    owner_account,  # Add the sender account parameter
    auction_house_address  # Optional: directly mint to the auction house
)

print(f"Minted token with ID: {token_id}")

Failed to get token ID from mint transaction
Minted token with ID: None


In [43]:
import requests
import json
from datetime import datetime

def get_auction_items(subgraph_url, auction_house_address):
    """
    Query an auction house's auctions from the subgraph.
    
    Args:
        subgraph_url: URL of the subgraph endpoint
        auction_house_address: Ethereum address of the auction house
        
    Returns:
        List of auctions with their details
    """
    query = """
    {
      auctionHouse(id: "%s") {
        name
        auctions {
          id
          auctionId
          tokenId
          tokenContract
          status
          reservePrice
          highestBidAmount
          currentBidder
          endTime
          startTime
          auctionOwner
          isPremiumAuction
          premiumBps
          tokenReference {
            metadata {
              name
              description
              image
            }
          }
          bids(orderBy: timestamp, orderDirection: desc, first: 1) {
            bidder
            amount
            timestamp
          }
        }
      }
    }
    """ % auction_house_address.lower()
    
    response = requests.post(subgraph_url, json={"query": query})
    response.raise_for_status()
    result = response.json()
    
    if "errors" in result:
        raise Exception(f"GraphQL Error: {result['errors']}")
    
    # Extract the auctions from the auction house
    auction_house = result["data"]["auctionHouse"]
    
    if auction_house:
        auctions = auction_house["auctions"]
        formatted_auctions = []
        
        for auction in auctions:
            # Format timestamps to human-readable format
            if auction["endTime"]:
                end_time = datetime.fromtimestamp(int(auction["endTime"]))
                end_time_str = end_time.strftime("%Y-%m-%d %H:%M:%S")
            else:
                end_time_str = "N/A"
                
            if auction["startTime"]:
                start_time = datetime.fromtimestamp(int(auction["startTime"]))
                start_time_str = start_time.strftime("%Y-%m-%d %H:%M:%S")
            else:
                start_time_str = "N/A"
            
            # Determine current winner
            winner = None
            if auction["status"] == "COMPLETED":
                winner = auction["currentBidder"]
                
            highest_bidder = None
            if auction["bids"] and len(auction["bids"]) > 0:
                highest_bidder = auction["bids"][0]["bidder"]
            
            # Create formatted auction object
            formatted_auction = {
                "id": auction["auctionId"],
                "status": auction["status"],
                "tokenId": auction["tokenId"],
                "tokenContract": auction["tokenContract"],
                "reservePrice": auction["reservePrice"],
                "highestBid": auction["highestBidAmount"],
                "currentBidder": highest_bidder,
                "winner": winner,
                "startTime": start_time_str,
                "endTime": end_time_str,
                "auctionOwner": auction["auctionOwner"],
                "isPremiumAuction": auction["isPremiumAuction"],
                "premiumRate": auction["premiumBps"]
            }
            
            # Add token metadata if available
            if auction["tokenReference"] and auction["tokenReference"]["metadata"]:
                metadata = auction["tokenReference"]["metadata"]
                formatted_auction["metadata"] = {
                    "name": metadata["name"],
                    "description": metadata["description"],
                    "image": metadata["image"]
                }
                
            formatted_auctions.append(formatted_auction)
            
        print(f"Found {len(formatted_auctions)} auctions for auction house {auction_house_address}")
        return formatted_auctions
    else:
        print(f"Auction house {auction_house_address} not found")
        return []


SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump-affiliate-review-curation/version/latest"
auction_house_address = "0xa753377e8a2012712faa05bdc74e99ef5f7e9767"

auctions = get_auction_items(SUBGRAPH_URL, auction_house_address)

# Print sample auction
if auctions:
    print("Sample auction:", json.dumps(auctions[0], indent=2))

Found 2 auctions for auction house 0xa753377e8a2012712faa05bdc74e99ef5f7e9767
Sample auction: {
  "id": "0",
  "status": "ACTIVE",
  "tokenId": "1",
  "tokenContract": "0xa72f04a5788530ac626eeb197f71b8938038990d",
  "reservePrice": "1000",
  "highestBid": "620250001102",
  "currentBidder": "0x988b8c3ad5971f79d1a461d17430afc5f1002eaa",
  "winner": null,
  "startTime": "2025-05-05 22:05:53",
  "endTime": "2025-05-05 22:21:53",
  "auctionOwner": "0xe360ee830bb3df8ab17c277f1802a2e861850b9b",
  "isPremiumAuction": true,
  "premiumRate": 0,
  "metadata": {
    "name": "Auction Item #1",
    "description": "This is a unique digital item being auctioned",
    "image": "https://assets.surlatable.com/m/15a89c2d9c6c1345/72_dpi_webp-REC-283110_Pizza.jpg"
  }
}


Ending Auction 1...
Error ending auction: 'str' object has no attribute 'functions'
